### 我们有一个组生存情况信息统计数据
#### 数据来源：https://www.kaggle.com/c/titanic/data
#### 问题：查看数据信息，求平均年龄。
#### 求男女生存率，未成年人获救的概率。

In [13]:
# coding=utf-8
import numpy as np
import pandas as pd
#中文的话这样打开，不会出现Initializing from file failed这种错误
f=open("./data/train.csv")
titanic=pd.read_csv(f)
titanic.head()

,survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,1,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,1,female,35.0,1,0,53.1000,First,C,Southampton,n
4,0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y


In [14]:
age = titanic["age"]
age_is_null = pd.isnull(age)#年龄是空值的布尔值
#如果不去除缺失值，就无法计算平均年龄
good_age = titanic["age"][age_is_null==False]
print(sum(good_age)/len(good_age)) #年龄的平均值
print(titanic["age"].mean()) #调用.mean()也可以直接计算年龄的平均值

29.631307814992027
29.631307814992027
